In [1]:
import json
import time
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from abc import ABC, abstractmethod
from enum import Enum
import threading
from queue import Queue, Empty

In [2]:
class MessageType(Enum):
    TASK_REQUEST = "task_request"
    TASK_RESPONSE = "task_response"
    COORDINATION = "coordination"
    STATUS_UPDATE = "status_update"

@dataclass
class Message:
    sender: str
    receiver: str
    message_type: MessageType
    content: Dict[str, Any]
    timestamp: float
    message_id: str

In [3]:
class SharedMemory:
    """Shared memory system for agents to store and retrieve information"""
    def __init__(self):
        self.data = {}
        self.lock = threading.Lock()
    
    def store(self, key: str, value: Any):
        with self.lock:
            self.data[key] = {
                'value': value,
                'timestamp': time.time()
            }
    
    def retrieve(self, key: str) -> Optional[Any]:
        with self.lock:
            return self.data.get(key, {}).get('value')
    
    def list_keys(self) -> List[str]:
        with self.lock:
            return list(self.data.keys())

In [4]:
class MessageBus:
    """Central message passing system for agent communication"""
    def __init__(self):
        self.agents = {}
        self.message_history = []
    
    def register_agent(self, agent_id: str, message_queue: Queue):
        self.agents[agent_id] = message_queue
    
    def send_message(self, message: Message):
        self.message_history.append(message)
        if message.receiver == "broadcast":
            # Broadcast to all agents except sender
            for agent_id, queue in self.agents.items():
                if agent_id != message.sender:
                    queue.put(message)
        elif message.receiver in self.agents:
            self.agents[message.receiver].put(message)
        else:
            print(f"Warning: Agent {message.receiver} not found")

In [5]:
class BaseAgent(ABC):
    """Base class for all agents in the system"""
    def __init__(self, agent_id: str, message_bus: MessageBus, shared_memory: SharedMemory):
        self.agent_id = agent_id
        self.message_bus = message_bus
        self.shared_memory = shared_memory
        self.message_queue = Queue()
        self.running = False
        self.thread = None
        
        # Register with message bus
        self.message_bus.register_agent(agent_id, self.message_queue)
    
    def start(self):
        """Start the agent in a separate thread"""
        self.running = True
        self.thread = threading.Thread(target=self._run)
        self.thread.start()
    
    def stop(self):
        """Stop the agent"""
        self.running = False
        if self.thread:
            self.thread.join()
    
    def _run(self):
        """Main agent loop"""
        while self.running:
            try:
                message = self.message_queue.get(timeout=0.1)
                self.handle_message(message)
            except Empty:
                continue
    
    def send_message(self, receiver: str, message_type: MessageType, content: Dict[str, Any]):
        """Send a message to another agent"""
        message = Message(
            sender=self.agent_id,
            receiver=receiver,
            message_type=message_type,
            content=content,
            timestamp=time.time(),
            message_id=f"{self.agent_id}_{int(time.time() * 1000)}"
        )
        self.message_bus.send_message(message)
    
    @abstractmethod
    def handle_message(self, message: Message):
        """Handle incoming messages"""
        pass
    
    @abstractmethod
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Process a specific task"""
        pass

In [6]:
class PlannerAgent(BaseAgent):
    """Agent responsible for task planning and decomposition"""
    
    def handle_message(self, message: Message):
        if message.message_type == MessageType.TASK_REQUEST:
            task_desc = message.content.get('description', 'Unknown')
            print(f"[{self.agent_id}] Received planning request: {task_desc}")
            result = self.process_task(message.content)
            # Include task_id in response
            result['task_id'] = message.content.get('task_id')
            self.send_message(
                message.sender,
                MessageType.TASK_RESPONSE,
                result
            )
    
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Create a plan for the given task"""
        task_description = task.get('description', '')
        
        # Simulate LLM planning logic
        if 'research' in task_description.lower():
            plan = {
                'task_id': task.get('task_id'),
                'plan': [
                    {'step': 1, 'action': 'gather_information', 'agent': 'researcher'},
                    {'step': 2, 'action': 'summarize_findings', 'agent': 'summarizer'},
                    {'step': 3, 'action': 'answer_questions', 'agent': 'answerer'}
                ],
                'estimated_time': 30,
                'resources_needed': ['web_access', 'knowledge_base']
            }
        elif 'analyze' in task_description.lower():
            plan = {
                'task_id': task.get('task_id'),
                'plan': [
                    {'step': 1, 'action': 'data_preprocessing', 'agent': 'researcher'},
                    {'step': 2, 'action': 'perform_analysis', 'agent': 'answerer'},
                    {'step': 3, 'action': 'create_summary', 'agent': 'summarizer'}
                ],
                'estimated_time': 45,
                'resources_needed': ['data_access', 'analysis_tools']
            }
        else:
            plan = {
                'task_id': task.get('task_id'),
                'plan': [
                    {'step': 1, 'action': 'understand_request', 'agent': 'answerer'},
                    {'step': 2, 'action': 'generate_response', 'agent': 'answerer'}
                ],
                'estimated_time': 15,
                'resources_needed': ['knowledge_base']
            }
        
        # Store plan in shared memory
        self.shared_memory.store(f"plan_{task.get('task_id')}", plan)
        print(f"[{self.agent_id}] Created plan with {len(plan['plan'])} steps")
        
        return {
            'status': 'success',
            'plan': plan,
            'message': f"Plan created with {len(plan['plan'])} steps"
        }

In [7]:
class ResearcherAgent(BaseAgent):
    """Agent responsible for information gathering and research"""
    
    def handle_message(self, message: Message):
        if message.message_type == MessageType.TASK_REQUEST:
            topic = message.content.get('topic', message.content.get('description', 'Unknown'))
            print(f"[{self.agent_id}] Received research request: {topic}")
            result = self.process_task(message.content)
            # Include task_id in response
            result['task_id'] = message.content.get('task_id')
            self.send_message(
                message.sender,
                MessageType.TASK_RESPONSE,
                result
            )
    
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Gather information for the given task"""
        topic = task.get('topic', task.get('description', ''))
        
        # Simulate research process
        time.sleep(1)  # Simulate processing time
        
        # More realistic mock research results based on topic
        if 'artificial intelligence' in topic.lower() or 'ai' in topic.lower():
            research_data = {
                'topic': topic,
                'sources': [
                    {'title': 'Large Language Models and Their Applications in 2024', 'relevance': 0.95},
                    {'title': 'Transformer Architecture Advances and Multimodal AI', 'relevance': 0.90},
                    {'title': 'AI Ethics and Responsible Development Frameworks', 'relevance': 0.85}
                ],
                'key_findings': [
                    'Large Language Models continue to show remarkable capabilities in reasoning and code generation',
                    'Multimodal AI systems combining text, image, and audio are becoming mainstream',
                    'AI safety and alignment research is gaining increased industry focus',
                    'Edge AI deployment is growing rapidly for real-time applications'
                ],
                'data_quality': 'high',
                'confidence': 0.92
            }
        elif 'remote work' in topic.lower() or 'productivity' in topic.lower():
            research_data = {
                'topic': topic,
                'sources': [
                    {'title': 'Remote Work Productivity Study 2024: Global Survey Results', 'relevance': 0.93},
                    {'title': 'Hybrid Work Models and Employee Satisfaction Analysis', 'relevance': 0.88},
                    {'title': 'Digital Collaboration Tools Impact on Team Performance', 'relevance': 0.82}
                ],
                'key_findings': [
                    'Remote workers report 13-15% higher productivity compared to pre-pandemic levels',
                    'Hybrid work models show optimal balance of collaboration and focus time',
                    'Investment in digital collaboration tools correlates with team performance',
                    'Employee work-life balance significantly improved with flexible arrangements'
                ],
                'data_quality': 'high',
                'confidence': 0.89
            }
        elif 'machine learning' in topic.lower() or 'ml' in topic.lower():
            research_data = {
                'topic': topic,
                'sources': [
                    {'title': 'Machine Learning Fundamentals: A Comprehensive Overview', 'relevance': 0.96},
                    {'title': 'Deep Learning vs Traditional ML: Comparative Analysis', 'relevance': 0.91},
                    {'title': 'ML Model Deployment and MLOps Best Practices', 'relevance': 0.87}
                ],
                'key_findings': [
                    'Supervised learning remains the most widely used ML paradigm in industry',
                    'Deep learning excels in unstructured data tasks like vision and NLP',
                    'Feature engineering and data quality are critical success factors',
                    'MLOps practices are essential for production ML system reliability'
                ],
                'data_quality': 'high',
                'confidence': 0.94
            }
        else:
            research_data = {
                'topic': topic,
                'sources': [
                    {'title': f'Comprehensive Analysis of {topic}', 'relevance': 0.85},
                    {'title': f'Recent Developments in {topic}', 'relevance': 0.80},
                    {'title': f'Industry Perspectives on {topic}', 'relevance': 0.75}
                ],
                'key_findings': [
                    f'Current research shows significant developments in {topic}',
                    f'Industry experts highlight key challenges in {topic}',
                    f'Future outlook for {topic} remains promising with ongoing innovations'
                ],
                'data_quality': 'medium',
                'confidence': 0.75
            }
        
        # Store research data in shared memory
        self.shared_memory.store(f"research_{task.get('task_id', 'unknown')}", research_data)
        print(f"[{self.agent_id}] Completed research on: {topic}")
        
        return {
            'status': 'success',
            'research_data': research_data,
            'message': f"Research completed for topic: {topic}"
        }


In [8]:
class SummarizerAgent(BaseAgent):
    """Agent responsible for summarizing information and findings"""
    
    def handle_message(self, message: Message):
        if message.message_type == MessageType.TASK_REQUEST:
            print(f"[{self.agent_id}] Received summarization request")
            result = self.process_task(message.content)
            # Include task_id in response
            result['task_id'] = message.content.get('task_id')
            self.send_message(
                message.sender,
                MessageType.TASK_RESPONSE,
                result
            )
    
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Create a summary of the given information"""
        task_id = task.get('task_id', 'unknown')
        
        # Retrieve research data from shared memory
        research_data = self.shared_memory.retrieve(f"research_{task_id}")
        
        if not research_data:
            return {
                'status': 'error',
                'message': 'No research data found to summarize'
            }
        
        # Simulate summarization process
        time.sleep(0.5)
        
        # Create more detailed summary
        topic_lower = research_data.get('topic', '').lower()
        
        if 'artificial intelligence' in topic_lower or 'ai' in topic_lower:
            executive_summary = "Current AI trends show rapid advancement in large language models, multimodal systems, and practical applications across industries, with growing emphasis on safety and ethical deployment."
        elif 'remote work' in topic_lower or 'productivity' in topic_lower:
            executive_summary = "Remote work analysis indicates sustained productivity gains, improved work-life balance, and the emergence of hybrid models as the preferred approach for most organizations."
        elif 'machine learning' in topic_lower or 'ml' in topic_lower:
            executive_summary = "Machine learning concepts encompass supervised, unsupervised, and reinforcement learning paradigms, with deep learning providing breakthrough capabilities for complex pattern recognition tasks."
        else:
            executive_summary = f"Analysis of {research_data.get('topic')} reveals several key insights and important considerations for stakeholders."
        summary = {
            'topic': research_data.get('topic', 'Unknown'),
            'executive_summary': executive_summary,
            'main_points': research_data.get('key_findings', []),
            'source_count': len(research_data.get('sources', [])),
            'confidence_level': research_data.get('confidence', 0.0),
            'recommendations': self._generate_recommendations(research_data.get('topic', ''))
        }
        
        # Store summary in shared memory
        self.shared_memory.store(f"summary_{task_id}", summary)
        print(f"[{self.agent_id}] Created summary for: {research_data.get('topic')}")
        
        return {
            'status': 'success',
            'summary': summary,
            'message': f"Summary created for {research_data.get('topic')}"
        }
    
    def _generate_recommendations(self, topic: str) -> List[str]:
        """Generate topic-specific recommendations"""
        topic_lower = topic.lower()
        
        if 'artificial intelligence' in topic_lower or 'ai' in topic_lower:
            return [
                "Stay updated on latest LLM developments and capabilities",
                "Consider ethical implications and bias mitigation strategies",
                "Explore practical applications in your specific domain",
                "Invest in AI safety and alignment research"
            ]
        elif 'remote work' in topic_lower or 'productivity' in topic_lower:
            return [
                "Implement hybrid work policies for optimal flexibility",
                "Invest in digital collaboration and communication tools",
                "Establish clear performance metrics for remote teams",
                "Prioritize employee well-being and work-life balance"
            ]
        elif 'machine learning' in topic_lower or 'ml' in topic_lower:
            return [
                "Focus on data quality and preprocessing techniques",
                "Choose appropriate algorithms based on problem type",
                "Implement proper model validation and testing procedures",
                "Consider MLOps practices for production deployment"
            ]
        else:
            return [
                f"Consider further investigation into {topic}",
                "Monitor recent developments in this area",
                "Validate findings with additional sources",
                "Consult with domain experts for deeper insights"
            ]


In [9]:
class AnswererAgent(BaseAgent):
    """Agent responsible for answering questions and providing responses"""
    
    def handle_message(self, message: Message):
        if message.message_type == MessageType.TASK_REQUEST:
            question = message.content.get('question', message.content.get('description', 'Unknown'))
            print(f"[{self.agent_id}] Received answer request: {question}")
            result = self.process_task(message.content)
            # Include task_id in response
            result['task_id'] = message.content.get('task_id')
            self.send_message(
                message.sender,
                MessageType.TASK_RESPONSE,
                result
            )
    
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Generate an answer based on available information"""
        task_id = task.get('task_id', 'unknown')
        question = task.get('question', task.get('description', ''))
        
        # Retrieve relevant information from shared memory
        research_data = self.shared_memory.retrieve(f"research_{task_id}")
        summary_data = self.shared_memory.retrieve(f"summary_{task_id}")
        
        # Generate more contextual answers
        if summary_data:
            # Create detailed answer based on summary
            topic = summary_data.get('topic', '').lower()
            main_points = summary_data.get('main_points', [])
            
            if 'artificial intelligence' in topic or 'ai' in topic:
                detailed_answer = f"{summary_data.get('executive_summary', '')} Key developments include: {', '.join(main_points[:2])}. This represents a significant shift in how AI systems are being developed and deployed."
            elif 'remote work' in topic or 'productivity' in topic:
                detailed_answer = f"{summary_data.get('executive_summary', '')} Research findings show: {', '.join(main_points[:2])}. These trends indicate a permanent shift in workplace dynamics."
            elif 'machine learning' in topic or 'ml' in topic:
                detailed_answer = f"{summary_data.get('executive_summary', '')} Core concepts include: {', '.join(main_points[:2])}. Understanding these fundamentals is crucial for effective ML implementation."
            else:
                detailed_answer = summary_data.get('executive_summary', 'Based on comprehensive analysis, several key insights emerge from the research.')
            
            answer = {
                'question': question,
                'answer': detailed_answer,
                'supporting_points': summary_data.get('main_points', []),
                'confidence': summary_data.get('confidence_level', 0.7),
                'sources_used': summary_data.get('source_count', 0),
                'recommendations': summary_data.get('recommendations', [])
            }
        elif research_data:
            # Generate answer from research data if no summary available
            topic = research_data.get('topic', '').lower()
            findings = research_data.get('key_findings', [])
            
            if 'artificial intelligence' in topic or 'ai' in topic:
                research_answer = f"Based on recent research into AI trends: {', '.join(findings[:2])}. These developments represent significant advances in the field."
            elif 'remote work' in topic or 'productivity' in topic:
                research_answer = f"Research on remote work productivity shows: {', '.join(findings[:2])}. These findings have important implications for organizational policies."
            elif 'machine learning' in topic or 'ml' in topic:
                research_answer = f"Machine learning research indicates: {', '.join(findings[:2])}. These principles form the foundation of effective ML systems."
            else:
                research_answer = f"Based on available research on {research_data.get('topic', 'this topic')}, several key insights emerge: {', '.join(findings[:2])}."
            
            answer = {
                'question': question,
                'answer': research_answer,
                'supporting_points': research_data.get('key_findings', []),
                'confidence': research_data.get('confidence', 0.7),
                'sources_used': len(research_data.get('sources', []))
            }
        else:
            # Fallback answer when no specific research is available
            if 'artificial intelligence' in question.lower() or 'ai' in question.lower():
                fallback_answer = "AI development encompasses machine learning, natural language processing, computer vision, and robotics, with recent focus on large language models and ethical AI practices."
            elif 'remote work' in question.lower() or 'productivity' in question.lower():
                fallback_answer = "Remote work has fundamentally changed workplace dynamics, generally showing productivity improvements while requiring new management approaches and digital tools."
            elif 'machine learning' in question.lower() or 'ml' in question.lower():
                fallback_answer = "Machine learning involves algorithms that learn patterns from data, including supervised learning (with labeled data), unsupervised learning (finding hidden patterns), and reinforcement learning (learning through interaction)."
            else:
                fallback_answer = f"I can provide general information about {question}, though specific research data would enhance the response quality."
            
            answer = {
                'question': question,
                'answer': fallback_answer,
                'supporting_points': [f"General knowledge about {question}"],
                'confidence': 0.6,
                'sources_used': 0
            }
        
        print(f"[{self.agent_id}] Generated answer for: {question}")
        
        return {
            'status': 'success',
            'answer': answer,
            'message': f"Answer generated for: {question}"
        }

In [10]:
class CoordinatorAgent(BaseAgent):
    """Central coordinator that manages task distribution and workflow"""
    
    def __init__(self, agent_id: str, message_bus: MessageBus, shared_memory: SharedMemory):
        super().__init__(agent_id, message_bus, shared_memory)
        self.active_tasks = {}
        self.task_counter = 0
    
    def handle_message(self, message: Message):
        if message.message_type == MessageType.TASK_RESPONSE:
            task_id = message.content.get('task_id')
            if task_id and task_id in self.active_tasks:
                print(f"[{self.agent_id}] Received response from {message.sender} for task {task_id}")
                self.process_task_response(task_id, message)
    
    def execute_task(self, description: str, question: str = None) -> Dict[str, Any]:
        """Execute a complete task using the multi-agent system"""
        self.task_counter += 1
        task_id = f"task_{self.task_counter}"
        
        print(f"\n[{self.agent_id}] Starting task execution: {description}")
        
        # Store task info
        self.active_tasks[task_id] = {
            'description': description,
            'question': question,
            'status': 'planning',
            'start_time': time.time(),
            'steps_completed': []
        }
        
        # Step 1: Get plan from planner
        self.send_message(
            'planner',
            MessageType.TASK_REQUEST,
            {
                'task_id': task_id,
                'description': description,
                'question': question
            }
        )
        
        # Wait for plan and execute steps
        return self.wait_for_completion(task_id)
    
    def process_task_response(self, task_id: str, message: Message):
        """Process responses from other agents"""
        task = self.active_tasks.get(task_id, {})
        sender = message.sender
        
        if sender == 'planner' and task.get('status') == 'planning':
            # Execute the plan
            plan = message.content.get('plan', {})
            task['plan'] = plan
            task['status'] = 'executing'
            task['steps_completed'].append('planning')
            print(f"[{self.agent_id}] Plan received, starting execution")
            self.execute_plan(task_id, plan)
        
        elif sender == 'researcher' and 'researching' in task.get('status', ''):
            task['steps_completed'].append('research')
            print(f"[{self.agent_id}] Research completed, proceeding to next step")
            self.continue_plan_execution(task_id, 'researcher')
        
        elif sender == 'summarizer' and 'summarizing' in task.get('status', ''):
            task['steps_completed'].append('summarization')
            print(f"[{self.agent_id}] Summarization completed, proceeding to next step")
            self.continue_plan_execution(task_id, 'summarizer')
        
        elif sender == 'answerer':
            task['status'] = 'completed'
            task['result'] = message.content
            task['end_time'] = time.time()
            task['steps_completed'].append('answering')
            print(f"[{self.agent_id}] Task {task_id} completed successfully")
    
    def execute_plan(self, task_id: str, plan: Dict[str, Any]):
        """Execute the first step of a plan"""
        steps = plan.get('plan', [])
        if steps:
            first_step = steps[0]
            agent = first_step.get('agent')
            
            if agent == 'researcher':
                self.active_tasks[task_id]['status'] = 'researching'
                self.send_message(
                    'researcher',
                    MessageType.TASK_REQUEST,
                    {
                        'task_id': task_id,
                        'topic': self.active_tasks[task_id]['description']
                    }
                )
            elif agent == 'answerer':
                self.active_tasks[task_id]['status'] = 'answering'
                self.send_message(
                    'answerer',
                    MessageType.TASK_REQUEST,
                    {
                        'task_id': task_id,
                        'question': self.active_tasks[task_id].get('question', 
                                                                 self.active_tasks[task_id]['description'])
                    }
                )
    
    def continue_plan_execution(self, task_id: str, completed_agent: str):
        """Continue executing the plan after an agent completes its task"""
        task = self.active_tasks.get(task_id, {})
        plan = task.get('plan', {})
        steps = plan.get('plan', [])
        
        # Find next step based on completed agent
        for i, step in enumerate(steps):
            if step.get('agent') == completed_agent:
                # Look for next step
                if i + 1 < len(steps):
                    next_step = steps[i + 1]
                    next_agent = next_step.get('agent')
                    
                    if next_agent == 'summarizer':
                        task['status'] = 'summarizing'
                        self.send_message(
                            'summarizer',
                            MessageType.TASK_REQUEST,
                            {'task_id': task_id}
                        )
                    elif next_agent == 'answerer':
                        task['status'] = 'answering'
                        self.send_message(
                            'answerer',
                            MessageType.TASK_REQUEST,
                            {
                                'task_id': task_id,
                                'question': task.get('question', task.get('description'))
                            }
                        )
                break
    
    def wait_for_completion(self, task_id: str, timeout: int = 30) -> Dict[str, Any]:
        """Wait for task completion and return results"""
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            task = self.active_tasks.get(task_id, {})
            if task.get('status') == 'completed':
                duration = task.get('end_time', time.time()) - task.get('start_time', start_time)
                print(f"\n[{self.agent_id}] Task completed in {duration:.2f} seconds")
                return {
                    'task_id': task_id,
                    'status': 'success',
                    'result': task.get('result', {}),
                    'duration': duration,
                    'steps_completed': task.get('steps_completed', [])
                }
            time.sleep(0.1)
        
        return {
            'task_id': task_id,
            'status': 'timeout',
            'message': 'Task did not complete within timeout period',
            'current_status': self.active_tasks.get(task_id, {}).get('status', 'unknown'),
            'steps_completed': self.active_tasks.get(task_id, {}).get('steps_completed', [])
        }
    
    def process_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        """Not used directly - coordination happens through execute_task"""
        return {'status': 'success', 'message': 'Coordinator process_task called'}

In [11]:
class MultiAgentSystem:
    """Main system that orchestrates all agents"""
    
    def __init__(self):
        self.shared_memory = SharedMemory()
        self.message_bus = MessageBus()
        self.agents = {}
        
        # Create agents
        self.agents['planner'] = PlannerAgent('planner', self.message_bus, self.shared_memory)
        self.agents['researcher'] = ResearcherAgent('researcher', self.message_bus, self.shared_memory)
        self.agents['summarizer'] = SummarizerAgent('summarizer', self.message_bus, self.shared_memory)
        self.agents['answerer'] = AnswererAgent('answerer', self.message_bus, self.shared_memory)
        self.agents['coordinator'] = CoordinatorAgent('coordinator', self.message_bus, self.shared_memory)
    
    def start(self):
        """Start all agents"""
        print("Starting Multi-Agent System...")
        for agent in self.agents.values():
            agent.start()
        time.sleep(0.5)  # Allow agents to initialize
        print("All agents started successfully!")
    
    def stop(self):
        """Stop all agents"""
        print("Stopping Multi-Agent System...")
        for agent in self.agents.values():
            agent.stop()
        print("All agents stopped!")
    
    def execute_task(self, description: str, question: str = None):
        """Execute a task using the multi-agent system"""
        return self.agents['coordinator'].execute_task(description, question)
    
    def get_system_status(self):
        """Get status of all agents and shared memory"""
        return {
            'agents': list(self.agents.keys()),
            'shared_memory_keys': self.shared_memory.list_keys(),
            'message_history_count': len(self.message_bus.message_history)
        }

In [12]:
# Example usage and demonstration
def main():
    """Demonstrate the multi-agent system"""
    # Create and start the system
    mas = MultiAgentSystem()
    mas.start()
    
    try:
        print("\n" + "="*50)
        print("MULTI-AGENT SYSTEM DEMONSTRATION")
        print("="*50)
        
        # Task 1: Research task
        print("\n--- Task 1: Research Task ---")
        result1 = mas.execute_task(
            "Research artificial intelligence trends",
            "What are the current trends in AI development?"
        )
        print(f"Status: {result1.get('status')}")
        print(f"Duration: {result1.get('duration', 0):.2f}s")
        print(f"Steps: {', '.join(result1.get('steps_completed', []))}")
        if result1.get('status') == 'success':
            answer = result1.get('result', {}).get('answer', {})
            print(f"\nQuestion: {answer.get('question', 'N/A')}")
            print(f"Answer: {answer.get('answer', 'N/A')}")
            print(f"Confidence: {answer.get('confidence', 0):.0%}")
            print(f"Sources: {answer.get('sources_used', 0)}")
            
            # Show supporting points
            points = answer.get('supporting_points', [])
            if points:
                print("Key Findings:")
                for i, point in enumerate(points[:3], 1):
                    print(f"  {i}. {point}")
        
        # Task 2: Analysis task
        print("\n--- Task 2: Analysis Task ---")
        result2 = mas.execute_task(
            "Analyze the impact of remote work on productivity",
            "How has remote work affected workplace productivity?"
        )
        print(f"Status: {result2.get('status')}")
        print(f"Duration: {result2.get('duration', 0):.2f}s")
        print(f"Steps: {', '.join(result2.get('steps_completed', []))}")
        if result2.get('status') == 'success':
            answer = result2.get('result', {}).get('answer', {})
            print(f"\nQuestion: {answer.get('question', 'N/A')}")
            print(f"Answer: {answer.get('answer', 'N/A')}")
            print(f"Confidence: {answer.get('confidence', 0):.0%}")
            print(f"Sources: {answer.get('sources_used', 0)}")
        
        # Task 3: General question
        print("\n--- Task 3: General Question ---")
        result3 = mas.execute_task(
            "Explain machine learning concepts",
            "What are the key concepts in machine learning?"
        )
        print(f"Status: {result3.get('status')}")
        print(f"Duration: {result3.get('duration', 0):.2f}s")
        print(f"Steps: {', '.join(result3.get('steps_completed', []))}")
        if result3.get('status') == 'success':
            answer = result3.get('result', {}).get('answer', {})
            print(f"\nQuestion: {answer.get('question', 'N/A')}")
            print(f"Answer: {answer.get('answer', 'N/A')}")
            print(f"Confidence: {answer.get('confidence', 0):.0%}")
            print(f"Sources: {answer.get('sources_used', 0)}")
            
            # Show recommendations if available
            recs = answer.get('recommendations', [])
            if recs:
                print("Recommendations:")
                for i, rec in enumerate(recs[:3], 1):
                    print(f"  {i}. {rec}")
        
        # Show system status
        print("\n--- System Status ---")
        status = mas.get_system_status()
        print(f"System Status: {status}")
        
    finally:
        # Clean shutdown
        mas.stop()

if __name__ == "__main__":
    main()

Starting Multi-Agent System...
All agents started successfully!

MULTI-AGENT SYSTEM DEMONSTRATION

--- Task 1: Research Task ---

[coordinator] Starting task execution: Research artificial intelligence trends
[planner] Received planning request: Research artificial intelligence trends
[planner] Created plan with 3 steps
[coordinator] Received response from planner for task task_1
[coordinator] Plan received, starting execution
[researcher] Received research request: Research artificial intelligence trends
[researcher] Completed research on: Research artificial intelligence trends
[coordinator] Received response from researcher for task task_1
[coordinator] Research completed, proceeding to next step
[summarizer] Received summarization request
[summarizer] Created summary for: Research artificial intelligence trends
[coordinator] Received response from summarizer for task task_1
[coordinator] Summarization completed, proceeding to next step
[answerer] Received answer request: What are t